In [1]:
users3 = {"David": {"Imagine Dragons": 3, "Daft Punk": 5,
"Lorde": 4, "Fall Out Boy": 1},
"Matt": {"Imagine Dragons": 3, "Daft Punk": 4,
"Lorde": 4, "Fall Out Boy": 1},
"Ben": {"Kacey Musgraves": 4, "Imagine Dragons": 3,
"Lorde": 3, "Fall Out Boy": 1},
"Chris": {"Kacey Musgraves": 4, "Imagine Dragons": 4,
"Daft Punk": 4, "Lorde": 3, "Fall Out Boy": 1},
"Tori": {"Kacey Musgraves": 5, "Imagine Dragons": 4,
"Daft Punk": 5, "Fall Out Boy": 3}}

In [6]:
import math
def computeSimilarity(artist1, artist2, userRating):
    average = {}
    for k, v in userRating.items():
        average[k] = float(sum(v.values()))/len(v.values())
    
    divedend = 0
    devisor1 = 0
    devisor2 = 0
    
    for user, ratings in userRating.items():
        if artist1 in ratings.keys() and artist2 in ratings.keys():
            avg = average[user]
            divedend += (ratings[artist1] - avg)*(ratings[artist2] - avg)
            devisor1 += (ratings[artist1] - avg)**2
            devisor2 += (ratings[artist2] - avg)**2
            
    return divedend/(math.sqrt(devisor1) * math.sqrt(devisor2))

In [8]:
computeSimilarity("Lorde", "Daft Punk", users3)

0.7841149584671063

###  Weighted Slope One method

extention of chapter 2(class recommender)

In [40]:
import codecs
class recommender:
    def pearson(self, rating1, rating2):
        sum_xy, sum_x, sum_y, sum_x2, sum_y2 = 0,0,0,0,0
        n = 0
        for key in rating1:
            if key in rating2:
                n += 1
                x = rating1[key]
                y = rating2[key]
                sum_xy += x*y
                sum_x += x
                sum_y += y
                sum_x2 += x**2
                sum_y2 += y**2
        if n == 0:
            return 0
        denominator = sqrt(sum_x2 - (sum_x**2)/n) * sqrt(sum_y2 - (sum_y**2)/n)
        if denominator == 0:
            return 0
        else:
            return (sum_xy - (sum_x * sum_y)/n)/denominator
        
    def __init__(self, data, k=1, metric='pearson', n=5):
        '''intialize recommender
        if data is dictionary, ther recommender is intialized
        Fora all other data types of data, no initialization occurs
        k is the k value for k nearest neighbor
        metirc is which distance formula to use
        n is the maximum number of recommendations to make'''
        self.k = k
        self.n = n
        self.username2id = {}
        self.userid2name = {}
        self.productid2name = {}
        self.frequencies = {}
        self.deviations = {}
        # save the name of the metric
        self.metric = metric
        if self.metric == 'pearson':
            self.fn = self.pearson
        
        if type(data).__name__ =='dict':
            self.data = data
        
    def convertProductID2name(self, id):
        """give product id number return product name"""
        if id in self.productid2name:
            return self.productid2name[id]
        else:
            return id
    
    def userRatings(self, id, n):
        """Return n top ratings for user with id"""
        print ("Ratings for " + self.userid2name[id])
        ratings = self.data[id]
        print(len(ratings))
        ratings = list(ratings.items())
        ratings = [(self.convertProductID2name(k), v) for (k, v) in ratings]
            # finally sort and return
        ratings.sort(key=lambda artistTuple: artistTuple[1],
        reverse = True)
        ratings = ratings[:n]
        for rating in ratings:
            print("%s\t%i" % (rating[0], rating[1]))
            
    def loadBookDB(self, path=''):
        """loads the BX book dataset
        Path is where the BX files are located"""
        self.data = {}
        i = 0
        head = False
        f = codecs.open(path + "BX-Book-Ratings.csv", 'r', encoding = "ISO-8859-1")
        for line in f:
            if not head:
                head = True
                continue
            i+=1
            #separate line into fields
            fields = line.split(';')
            user = fields[0].strip('"')
            book = fields[1].strip('"')
            rating = fields[2].strip().strip('"')
            rating = int(rating)
            if user in self.data:
                currentRatings = self.data[user]
            else:
                currentRatings = {}
            currentRatings[book] = rating
            self.data[user] = currentRatings
        f.close()
        #
        # Now load books into self.productid2name
        # Books contains isbn, title, and author among other fields
        #
        f = codecs.open(path + "BX-Books.csv", 'r', encoding = "ISO-8859-1")
        head = False
        for line in f:
            if not head:
                head = True
                continue
            i += 1
            # separate line into fields
            fields = line.split(';')
            isbn = fields[0].strip('"')
            title = fields[1].strip('"')
            author = fields[2].strip().strip('"')
            title = title + ' by ' + author
            self.productid2name[isbn] = title
        f.close()
        f = codecs.open(path + "BX-Users.csv",'r', encoding = "ISO-8859-1")
        head = False
        for line in f:
            if not head:
                head = True
                continue
            i += 1
            #separate line into fields
            fields = line.split(';')
            userid = fields[0].strip('"')
            location = fields[1].strip('"')
            if len(fields) > 3:
                age = fields[2].strip().strip('"')
            else:
                age = 'NULL'
            if age != 'NULL':
                value = location +' (age:' + age + ')'
            else:
                value = location
            self.userid2name[userid] = value
            self.username2id[location] = userid
        f.close()
        print(i)
            

    def loadMovieLens(self, path=''):
        self.data = {}
        # first load movie ratings
        i = 0
        # First load book ratings into self.data
        #f = codecs.open(path + "u.data", 'r', 'utf8')
        f = codecs.open(path + "u.data", 'r', 'ascii')
        #  f = open(path + "u.data")
        for line in f:
            i += 1
            #separate line into fields
            fields = line.split('\t')
            user = fields[0]
            movie = fields[1]
            rating = int(fields[2].strip().strip('"'))
            if user in self.data:
                currentRatings = self.data[user]
            else:
                currentRatings = {}
            currentRatings[movie] = rating
            self.data[user] = currentRatings
        f.close()
        #
        # Now load movie into self.productid2name
        # the file u.item contains movie id, title, release date among
        # other fields
        #f = codecs.open(path + "u.item", 'r', 'utf8')
        f = codecs.open(path + "u.item", 'r', 'iso8859-1', 'ignore')
        #f = open(path + "u.item")
        for line in f:
            i += 1
            #separate line into fields
            fields = line.split('|')
            mid = fields[0].strip()
            title = fields[1].strip()
            self.productid2name[mid] = title
        f.close()
        #  Now load user info into both self.userid2name
        #  and self.username2id
        #f = codecs.open(path + "u.user", 'r', 'utf8')
        f = open(path + "u.user")
        for line in f:
            i += 1
            fields = line.split('|')
            userid = fields[0].strip('"')
            self.userid2name[userid] = line
            self.username2id[line] = userid
        f.close()
        print(i)
            
    def computeNearestNeighbor(self, username):
        """creates a sorted list of users based on their distance to            
        username"""
        distances = []
        for instance in self.data:
            if instance != username:
                distance = self.fn(self.data[username],self.data[instance])
                distances.append((instance, distance))
        # sort based on distance -- closest first
        distances.sort(key=lambda artistTuple: artistTuple[1],reverse=True)
        return distances
    
    def recommend(self, user):
        recommendations = {}
        #get list of users ordered by nearness
        nearest = self.computeNearestNeighbor(user)            
        # now get the ratings for the user
        userRatings = self.data[user]
        ##
        # determine the total distance
        totalDistance = 0.0
        for i in range(self.k):
            totalDistance += nearest[i][1]
        # now iterate through the k nearest neighbors
        # accumulating their ratings
        for i in range(self.k):
            # compute slice of pie
            weight = nearest[i][1] / totalDistance
            # get the name of the person
            name = nearest[i][0]
            # get the ratings for this person
            neighborRatings = self.data[name]
            # get the name of the person
            # now find bands neighbor rated that user didn't
            for artist in neighborRatings:
                if not artist in userRatings:
                    if artist not in recommendations:
                        recommendations[artist] = (neighborRatings[artist] * weight)
                    else:
                        recommendations[artist] = (recommendations[artist] +\
                                                   neighborRatings[artist] * weight)
        recommendations = list(recommendations.items())
        recommendations = [(self.convertProductID2name(k),v) \
                              for (k, v) in recommendations]
        # finally sort and return
        recommendations.sort(key=lambda x:x[1],reverse =True)
        # Return the first n items
        return recommendations[:self.n]
    
    # new part by chapter 3
    def computeDeviations(self):
        # for each person in the data: get their ratings
        for ratings in self.data.values():
            # for each item & rating in that set of ratings
            for (item, rating) in ratings.items():
                self.frequencies.setdefault(item, {})
                self.deviations.setdefault(item, {})
                # for each item2 & rating2 in the set of ratings:
                for (item2, rating2) in ratings.items():
                    self.frequencies[item].setdefault(item2, 0)
                    self.deviations[item].setdefault(item2, 0.0)
                    self.frequencies[item][item2] += 1
                    self.deviations[item][item2] += rating - rating2
        for (item, ratings) in self.deviations.items():
            for item2 in ratings:
                #print(item, item2,ratings[item2], self.frequencies[item][item2] )
                ratings[item2] /= self.frequencies[item][item2]
                
    # new part by chapter 3
    def slopeOneRecommendations(self, userRatings):
        recommendations = {}
        frequencies = {}
        # for every item and rating in the user's recommendations
        for (userItem, userRating) in userRatings.items():
            # for every item in our datset that the user didn't rate
            for (diffItem, diffRatings) in self.deviations.items():
                if diffItem not in userRatings and \
                   userItem in self.deviations[diffItem]:
                    freq = self.frequencies[diffItem][userItem]
                    recommendations.setdefault(diffItem, 0.0)
                    frequencies.setdefault(diffItem, 0)
                    # add to the rolling sum representing the devidend
                    recommendations[diffItem] += (diffRatings[userItem] + userRating) * freq
                    # keep a rolling sum of the frequency of diff item
                    frequencies[diffItem] += freq
        recommendations = [(self.convertProductID2name(k), v/frequencies[k] )\
                           for(k,v) in recommendations.items()]
        return recommendations

In [14]:
users2 = {"Amy": {"Taylor Swift": 4, "PSY": 3, "Whitney Houston": 4},
"Ben": {"Taylor Swift": 5, "PSY": 2},
"Clara": {"PSY": 3.5, "Whitney Houston": 4},
"Daisy": {"Taylor Swift": 5, "Whitney Houston": 3}}

In [41]:
r = recommender(users2)
r.computeDeviations()
r.deviations

{'PSY': {'PSY': 0.0, 'Taylor Swift': -2.0, 'Whitney Houston': -0.75},
 'Taylor Swift': {'PSY': 2.0, 'Taylor Swift': 0.0, 'Whitney Houston': 1.0},
 'Whitney Houston': {'PSY': 0.75,
  'Taylor Swift': -1.0,
  'Whitney Houston': 0.0}}

In [42]:
r.slopeOneRecommendations(users2['Ben'])

[('Whitney Houston', 3.375)]